<a href="https://colab.research.google.com/github/fshivam/Automated_Paper_correction/blob/master/Automated_paper_correction_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# Install the latest Tensorflow version.
#!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
#!pip3 install tensorflow-hub
!pip3 install seaborn
!pip3 install fpdf==1.7.2

**Loading all the essential libraries and the Universal Sentence Encoder**

In [ ]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import time
import pandas as pd
import matplotlib.pyplot as plt

module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder-large/5 loaded


**The Sample Answer Key is as follows:**


"The issue of extending political rights to women was a controversial one within the liberal movement, in which large numbers of women had participated actively over the years. Women had formed their own political associations, founded newspapers and taken part in political meetings and demonstrations. Despite this, they were denied suffrage during the election of the Assembly. When the Frankfurt Parliament convened in the Church of St. Paul, women were admitted only as observers to stand in the visitors’ gallery." 



---


**The above answer key consists of *four* points**


But the number of points may not always be equal. For example, a student may split one point in two or combine two points in one or he may miss some points altogether. 

So for this demo, we'll consider the following two variations:



---



**The Sample Student Answer 1 is as follows:**


In Post-Medivial Germany, women were still largely thought of as nurturers. Consequently, the issue of extending political rights to women was a controversial one within the liberal movement. A large number of women had participated in the liberal movement over the years. In fact, women had formed their own political associations, founded newspapers and taken part in political meetings. They even participated in various political demonstrations. But despite this, they were denied voting rights during the election of Assembly. When the Frankfurt parliament convened in Church of St. Paul, women were only admitted as observers. 

**The above student answer consists of *seven* points**



---



**The Sample Student Answer 2 is as follows:**

"Many woman participated in the issue of demanding basic rights such as voting which was deemed controversial. But women had already formed their own political associations and even founded newspapers. Inspite of all this involvement, they were still denied voting rights."


**The above student answer consists of *three* points**



In [ ]:
# Define the sample answer key.
answer_key = ['The issue of extending political rights to women was a controversial one within the liberal movement, in which large numbers of women had participated actively over the years. Women had formed their own political associations, founded newspapers and taken part in political meetings and demonstrations. Despite this, they were denied suffrage during the election of the Assembly. When the Frankfurt Parliament convened in the Church of St. Paul, women were admitted only as observers to stand in the visitors’ gallery.']
answer_key2 = ['Two examples to show the contribution of culture to the growth of nationalism in Europe were: 1. Romanticism was a European cultural movement aimed at developing national unity by creating a sense of shared heritage and common history. The Romantic artists emphasis on emotions, intuition and mystical feelings gave shape and expression to nationalist sentiments. The strength of art in promoting nationalism is well exemplified in the role played by European poets and artists in mobilising public opinion to support the Greeks in their struggle to establish their national identity. 2.  Language also played a distinctive role in developing nationalist feelings in Europe. An example of this is how during Russian occupation, the use of Polish came to be seen as a symbol of struggle against Russian dominance. During this period, Polish language was forced out of schools and Russian language was imposed everywhere. Following the defeat of an armed rebellion against Russian rule in 1831, many members of the clergy in Poland began using language as a weapon of national resistance. They did so by refusing to preach in Russian, and by using Polish for Church gatherings and religious instruction. The emphasis on the use of vernacular language, the language of the masses, helped spread the message of national unity.']
answerkey_3 = ['Nationalist tensions .emerged in  . the Balkans because of the .spread of ideas of.. romantic nationalism as also the disintegration of the Ottoman Empire that had previously ruled over this area. The different Slavic communities in the Balkans began to strive for independent rule. They were jealous of each other and every state wanted more territory, even at the expense of others. Also, the hold of imperial power over the Balkans made the situation worse. Russia, Germany, England, Austro-Hungary all wanted more control over this area. These conflicts ultimately led to the First World War in 1914.']
# Define the variations that we'll be testing out for. 

variation_1 = ['Large numbers of women had participated actively over the years in the issue of demanding political rights such as voting. Although this was controversial, women had formed their political associations, founded newspapers and taken part in meetings. But they were still denied voting rights.']
variation_2 = ['Many woman participated in the issue of demanding basic rights such as voting which was deemed controversial. But women had already formed their own political associations and even founded newspapers. Inspite of all this involvement, they were still denied voting rights.']
variation_3 = ['The issue of extending political rights to women was controversial because at that time women were thought of only as nurturers even though they participated in political meetings, founded newspapers and formed their own political associations. Over time, large number of women had actively participated in the issue of demanding voting and other political rights.']
variation_4 = ['In Post-Medivial Germany, women were still largely thought of as nurturers. Consequently, the issue of extending political rights to women was a controversial one within the liberal movement. A large number of women had participated in the liberal movement over the years. In fact, women had formed their own political associations, founded newspapers and taken part in political meetings. They even participated in various political demonstrations. But despite this, they were denied voting rights during the election of Assembly. When the Frankfurt parliament convened in Church of St. Paul, women were only admitted as observers.']
variation_6 = ['In post-medivial Germany, women were not considered equal but only as nuruturers. So the issue of giving them voting rights was controversial. Because of this, their participation in the nationalist struggle was limited.']
variation_5 = ['The role of women in nationalist struggles was very limited. This was because they were denied voting and other political rights during the election of the assembly. Because of this, their role was limited as mentioned previously inspite of having found newspapers, formed political associations, and even participating in meetings and demonstrations.']
variation_7 = ["Two examples to show the contribution of culture to the growth of nationalism : 1. A primory factor for the growth of nationalism in Europe was Romanticism Romanticism was a European culture movement aimed at developing national unity by creating a sense of shared heritage and common history, The Romantica artist's emphasis on emotions intuition and mystical feelings gave shape and expression to nationalist feelings. 2. Apart from Romanticism, language also played an important role in developing nationalist feeling in Europe. For example, during Russian occupation, the use of Polish was seen as a symbol of struggle against Russions. During this period, Polish language was forced out of schools and Russian was made compulsory everywhere. Mony members of the clerzy started refusing to preach in Russian, and used Polish for church gatherings and other religious activities. This way & use the of Polish helped spread the message of national unity, se on whenua-"]
variation_8 = ['Patriotic strains developed in the Balkans because of the spread of ideas of romantic nationalism as also the breaking down of the Ottoman Empire that had recently administered over this territory. All the different Slavic communities in the Balkans werd envious of one another and cach state wanted more area. Also, the hold of imperial power over the Balkans me worsened the situation. Russia, Germany, England , Asustro-Hungary all wonted more control over this zone which brought about the first world war in 1914.']
v9 = ["Two examples to show the contribution of culture to the growth of nationalism : A primary factor for the growth of nationalism in Europe was Romanticism Romanticism was a European culture. movement aimed at developing national unity by creating a sense of shared heritage and common history The Romantic artist's emphasis on emotions intuition and mystical feelings gave shape and expression to nationalist feelings. 2. Apart from romanticism language also played an important role in developing nationalist feeling in Europe. For example during was seen as a symbol of struggle against russians during this period Polish language was forced out of schools and Russian was made compulsory everywhere Mony members of the clergy started refusing to preach in russian and used Polish for church gathtus. This way there of Polish helped spread the ... message of national unity"]

In [ ]:
# Reduce logging output. 
logging.set_verbosity(logging.ERROR)

# Import nltk 
import nltk
nltk.download('punkt')
from nltk.tokenize import TweetTokenizer

# Convert the answer into a list of sentences
def convert_answers_to_sentences(text):
  sent_text = nltk.sent_tokenize(text)
  # Return a list of sentences from the passed text
  return sent_text 


# Convert the split sentences into a list words.
def convert_sentences_to_words(sent_text):
  for sentence in sent_text:
    tokenized_text = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokenized_text)
    print(tagged)

    # tokenizer_words = TweetTokenizer()
    # tokens_sentences = [tokenizer_words.tokenize(t) for t in sent_text]
    # print(tokens_sentences)
    # return tokens_sentences

# Convert the answer into a list sentences with words segmented.
def convert_answers_to_words(text):
  tokenizer_words = TweetTokenizer()
  tokens_sentences = [tokenizer_words.tokenize(t) for t in nltk.sent_tokenize(input_text)]
  print(tokens_sentences)    
  return tokens_sentences


def delete_erraneous_sentences(ans_stu_sent):

  for idx, sent in enumerate(ans_stu_sent):
    if len(sent.split()) == 1:
      del(ans_stu_sent[idx])
  return ans_stu_sent

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Import cosine similarity evaluation metric

from sklearn.metrics.pairwise import cosine_similarity


# Calculate embeddings of the sentences in the answer key
def get_embeddings(a_list_of_sentences):
  sentence_embeddings = []
  sentence_embeddings.append(embed(a_list_of_sentences))
  return sentence_embeddings


# Convert tf.EagerTensor to ndarray 
def convert_eagertensor_to_ndarray(eagertensor):
  ndarray = np.empty((len(eagertensor[0]), 512))
  for i in range(len(eagertensor[0])):
    ndarray[i,:] = np.asarray(eagertensor[0][i])
  return ndarray
  


# Calculate similarity matrix 
def get_similarity_matrix(question_paper_embeddings, answer_key_embeddings):
  sim_mat = np.empty((len(answer_key_embeddings), len(question_paper_embeddings)))
  for ans_sen_number, anskey_embedding in enumerate(answer_key_embeddings):
      anskey_embedding = anskey_embedding.reshape(1,-1)
      for stu_sen_number, stu_embedding in enumerate(question_paper_embeddings):
        stu_embedding = stu_embedding.reshape(1,-1)
        sim_mat[ans_sen_number, stu_sen_number] = cosine_similarity(anskey_embedding, stu_embedding)
  return sim_mat    

# Check number of sentences 

def check_num_sentences(sentence):
  sent = convert_answers_to_sentences(sentence)
  return (len(sent))


In [ ]:
def interpret_sim_mat_and_generate_report(sim_mat, sim_mat_trans):


  # Open a file for dumping text
  f = open("/tmp/output.txt", "a", encoding='latin-1')

  


  #print(color.RED + 'x------------------------ Student Feedback Report ------------------------x \n' + color.END, file=f)
  #print('\n', file=f)
  #print('\n', file=f)



  # Feedback
  ind = np.argmax(sim_mat, axis=1)
  strong_semantic_match = []
  loose_semantic_match = []
  no_semantic_match = []

  
  for i in range(len(ind)):
    similarity = sim_mat[i][ind[i]]
    if similarity < 0.35:
      no_semantic_match.append([i,similarity])
    elif similarity > 0.35 and similarity <= 0.65:
       loose_semantic_match.append([i,similarity])   
    else:
      strong_semantic_match.append([i,similarity]) 
  

  
  print('Student has not written anything covering the following points: \n', file=f)
  print('\n', file=f)  
  if no_semantic_match!=[]: 
    for i in range(len(no_semantic_match)):  
      print(ans_key_sent[no_semantic_match[i][0]], file=f)
  elif no_semantic_match==[]:
    print('None. All points were adequately covered \n', file=f)  

  print('------------------------------------------------------------------- \n', file=f)
  print('\n', file=f)  
  print('\n', file=f)
    


  print('------------------------------------------------------------------- \n', file=f)
  print('Student has loosely touched upon the following points, but a more adequate coverage was expected: \n', file=f)
  print('\n', file=f)  
  
  if loose_semantic_match!=[]:
    for i in range(len(loose_semantic_match)):  
      print(ans_key_sent[loose_semantic_match[i][0]], file=f)
  elif loose_semantic_match==[]:
    print('None. All points were adequately covered \n', file=f)  

  print('------------------------------------------------------------------- \n', file=f)
  print('\n', file=f)  
  print('\n', file=f)


  print('------------------------------------------------------------------- \n', file=f)
  print('The student has covered the following points perfectly and as per expectation: \n', file=f)
  print('\n', file=f)  

  if strong_semantic_match!=[]:
    for i in range(len(strong_semantic_match)):  
      print(ans_key_sent[strong_semantic_match[i][0]], file=f)
  elif strong_semantic_match==[]:
    print('None of the expected points were adequately covered \n', file=f)  

  print('------------------------------------------------------------------- \n', file=f)
  print('\n', file=f)  
  print('\n', file=f)

  # ------- Student point of view --------- # 

  ind = np.argmax(sim_mat_trans, axis=1)
  #print(ind)
  no_semantic_match = []
  loose_semantic_match = []
  strong_semantic_match = []
  assigned_weights = {}

  for i in range(len(ind)):
      similarity = sim_mat_trans[i][ind[i]]
      if similarity < 0.35: 
        no_semantic_match.append([i, similarity])
      elif similarity > 0.35 and similarity <= 0.65:
        loose_semantic_match.append([i, similarity])
      else:
        strong_semantic_match.append([i, similarity])
          
      
  print('------------------------------------------------------------------- \n', file=f)
  print('The points that loosely matched the answer key were: \n', file=f)
  print('\n', file=f)  

  if loose_semantic_match!=[]:
    for i in range(len(loose_semantic_match)):          
      print(ans_stu_sent[loose_semantic_match[i][0]], file=f)
      assigned_weights[ans_stu_sent[loose_semantic_match[i][0]]] = loose_semantic_match[i][1]
  elif loose_semantic_match==[]:
    print('No such sentences \n', file=f)

  print('------------------------------------------------------------------- \n', file=f)  
  print('\n', file=f)  
  print('\n', file=f)


  print('------------------------------------------------------------------- \n', file=f)
  print('The points that strongly matched the answer key were: \n', file=f)
  print('\n', file=f)  

  if strong_semantic_match!=[]:
    for i in range(len(strong_semantic_match)):          
      print(ans_stu_sent[strong_semantic_match[i][0]], file=f)
      assigned_weights[ans_stu_sent[strong_semantic_match[i][0]]] = 1.0
  elif strong_semantic_match==[]:
    print('No such points \n', file=f)

  print('------------------------------------------------------------------- \n', file=f)
  print('\n', file=f)  
  print('\n', file=f)


  print('------------------------------------------------------------------- \n', file=f)  
  print('The sentences that were extra and did not find a match in the the answer key were: \n', file=f)
  print('\n', file=f)  
  
  if no_semantic_match!=[]:
    for i in range(len(no_semantic_match)):          
      print(ans_stu_sent[no_semantic_match[i][0]], file=f)
      assigned_weights[ans_stu_sent[no_semantic_match[i][0]]] = 0.0
  elif no_semantic_match==[]:
    print('No such sentences \n', file=f)  

  print('------------------------------------------------------------------- \n', file=f)
  print('\n', file=f)  
  print('\n', file=f)

  return assigned_weights, ans_stu_sent, f

In [ ]:
# Semantically Normalise Sentences

def normalise_sentences(ans_key_sent, ans_stu_sent, sim_mat, sim_mat_trans):
  modifications = []
  thresh = 0.45
  ind = np.argmax(sim_mat, axis=1)
  ind_trans = np.argmax(sim_mat_trans, axis=1)
  print(ind_trans)
  for i in range(len(ind)):
    similarity = sim_mat[i][ind[i]]
    if similarity>=0.35:
      new_similarity = similarity*thresh 
      for idx, j in enumerate(sim_mat[i]):
        if j>= new_similarity and j<similarity:
          if j==sim_mat_trans[idx][ind_trans[idx]]:
            modifications.append([i, idx, j, ind[i], ans_stu_sent[idx], ans_stu_sent[ind[i]]])
    else:
      continue      
  print(modifications)      



  if modifications!=[]:
    for i in modifications:
      comb = i[5] + ' ' + i[4]
      ans_stu_sent[i[3]] = comb
      ans_stu_sent[i[1]] = ''  
  else:
    print('No possible combinations found.')

  if modifications!=[]:
    for idx, sentences in enumerate(ans_stu_sent):
      if sentences=='':
        ans_stu_sent.pop(idx)
  else:
    print('No modifications necessary!')      
  
  ########################

  modifications2 = []
  thresh = 0.45
  ind = np.argmax(sim_mat, axis=1)
  ind_trans = np.argmax(sim_mat_trans, axis=1)
  print(ind_trans)
  for i in range(len(ind_trans)):
    similarity = sim_mat_trans[i][ind_trans[i]]
    if similarity>=0.35:
      new_similarity = similarity*thresh 
      for idx, j in enumerate(sim_mat_trans[i]):
        if j>= new_similarity and j<similarity:
          if j==sim_mat[idx][ind[idx]]:
            modifications2.append([i, idx, j, ind_trans[i], ans_key_sent[idx], ans_key_sent[ind_trans[i]]])
    else:
      continue      
  print(modifications2)      

  

  if modifications2!=[]:
    for i in modifications2:
      comb = i[5] + ' ' + i[4]
      ans_key_sent[i[3]] = comb
      ans_key_sent[i[1]] = ''  
  else:
    print('No possible combinations found.')

  if modifications2!=[]:
    for idx, sentences in enumerate(ans_key_sent):
      if sentences=='':
        ans_key_sent.pop(idx)
  else:
    print('No modifications necessary!')      

  return ans_stu_sent, ans_key_sent



# Denormalise Sentences

def denormalise_sentences(ans_stu_sent, ans_stu_main, assigned_weights):
  weights = {}
  for i in range(len(ans_stu_main)):
    for j in range(len(ans_stu_sent)):
      if ans_stu_main[i] in ans_stu_sent[j]:
        if len(ans_stu_main[i]) < len(ans_stu_sent[j]):  
          n = check_num_sentences(ans_stu_sent[j])
          weights[ans_stu_main[i]] = (assigned_weights[ans_stu_sent[j]])/n
        else:
          weights[ans_stu_main[i]] =  assigned_weights[ans_stu_sent[j]] 

  return weights      



In [ ]:
# Split answer key and student answer into sentences.
start_time = time.time()
ans_key_sent = convert_answers_to_sentences(answerkey_3[0])
ans_stu_sent = convert_answers_to_sentences(variation_9[0])
ans_stu_sent = delete_erraneous_sentences(ans_stu_sent)
ans_key_sent = delete_erraneous_sentences(ans_key_sent)

print(ans_key_sent)
print(ans_stu_sent)
print(len(ans_key_sent))
print(len(ans_stu_sent))

# Get embeddings for student answer and answer key 
question_paper_embeddings = get_embeddings(ans_stu_sent)
question_paper_embeddings = convert_eagertensor_to_ndarray(question_paper_embeddings)
answer_key_embeddings = get_embeddings(ans_key_sent)
answer_key_embeddings = convert_eagertensor_to_ndarray(answer_key_embeddings)


# Get similarity matrix (answer key sentences -> compared with -> student answer sentences)
sim_mat = get_similarity_matrix(question_paper_embeddings, answer_key_embeddings)

# Get similarity matrix (student answer sentences -> compared with -> answer key sentence)
sim_mat_trans = sim_mat.transpose()

print("--- %s seconds ---" % (time.time() - start_time))

['Nationalist tensions .emerged in  .', 'the Balkans because of the .spread of ideas of.. romantic nationalism as also the disintegration of the Ottoman Empire that had previously ruled over this area.', 'The different Slavic communities in the Balkans began to strive for independent rule.', 'They were jealous of each other and every state wanted more territory, even at the expense of others.', 'Also, the hold of imperial power over the Balkans made the situation worse.', 'Russia, Germany, England, Austro-Hungary all wanted more control over this area.', 'These conflicts ultimately led to the First World War in 1914.']
['It was a German National Assembly made upon middle class professionals businessmen artisans belonging to different German regiony.', 'It was convened on 18 may 1848 in the Church of St. Paul frankfurt The assembly was responsible for drafting constitution for Germany which for was two-headed by a monarchy ruler Be that as if may it confronted resistance from the privil

In [ ]:
# Get normalised sentences
ans_stu_main = list(ans_stu_sent)
ans_stu_sent, ans_key_sent = normalise_sentences(ans_key_sent, ans_stu_sent, sim_mat, sim_mat_trans)
question_paper_embeddings = get_embeddings(ans_stu_sent)
question_paper_embeddings = convert_eagertensor_to_ndarray(question_paper_embeddings)
answer_key_embeddings = get_embeddings(ans_key_sent)
answer_key_embeddings = convert_eagertensor_to_ndarray(answer_key_embeddings)

# Get similarity matrix (answer key sentences -> compared with -> student answer sentences)
sim_mat = get_similarity_matrix(question_paper_embeddings, answer_key_embeddings)

# Get similarity matrix (student answer sentences -> compared with -> answer key sentence)
sim_mat_trans = sim_mat.transpose()

[0 1 3 4]
[]
No possible combinations found.
No modifications necessary!
[0 1 3 4]
[[1, 2, 0.4663936737822499, 1, 'They were jealous of each other and every state wanted more territory, even at the expense of others.', 'The different Slavic communities in the Balkans began to strive for independent rule.'], [3, 5, 0.509601714769388, 4, 'These conflicts ultimately led to the First World War in 1914.', 'Russia, Germany, England, Austro-Hungary all wanted more control over this area.']]


**Using the vector embeddings and similarity matrix from the above line of code, we will now generate a report which includes feedback**

In [ ]:
assigned_weights, ans_stu_sent, f = interpret_sim_mat_and_generate_report(sim_mat, sim_mat_trans)

In [ ]:
weights = denormalise_sentences(ans_stu_sent, ans_stu_main, assigned_weights)


# To set each column cell to be just as big as it needs to be in order to fully display it's content.
#pd.set_option('display.max_colwidth', 0)
pd.options.display.max_colwidth = 50

TOTAL_MARKS = 2.0
marks_per_point = (TOTAL_MARKS)/len(ans_key_sent)


df = pd.DataFrame(weights.items(), columns=["Student's Point", 'Associated Weight'])
df['Marks Awarded'] = df['Associated Weight']
df['Marks Awarded'] *= marks_per_point
df["Student's Point"] = df["Student's Point"].str.wrap(50)

df.to_csv('/tmp/out.csv', index=False)

print('Evaluated Score: ' , file=f)
print(str(df['Marks Awarded'].sum()) + ' out of ' + str(TOTAL_MARKS), file=f)

# Close stream to text file
f.close()

# **This section is dedicated to the generation of the report from the above obtained console outputs. This section is still under development, all the updates will be posted here for the readers' as well as the author's own reference.** 

**Update 12th March, 2020:** Completely working code with report generation ready. 

Report generation time (5 page): 
62 seconds (without GPU)

**Update 25th March, 2020:** Added optimisations to boost the report generation time. 

Report generation time (5 page): 
2.7 seconds (without GPU)


In [ ]:
from fpdf import FPDF

title = 'Feedback Report'

class PDF(FPDF):
    def header(self):
        # Arial bold 15
        self.set_font('Arial', 'B', 16)
        # Calculate width of title and position
        w = self.get_string_width(title) + 6
        self.set_x((210 - w) / 2)
        # Colors of frame, background and text
        self.set_draw_color(0, 80, 180)
        self.set_fill_color(230, 230, 0)
        self.set_text_color(220, 50, 50)
        # Thickness of frame (1 mm)
        self.set_line_width(1)
        # Title
        self.cell(w, 9, title, 1, 1, 'C', 1)
        # Line break
        self.ln(10)

    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        # Arial italic 8
        self.set_font('Arial', 'I', 8)
        # Text color in gray
        self.set_text_color(128)
        # Page number
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    def titles(self, num, label):
        # Arial 12
        self.set_font('Arial', '', 12)
        # Background color
        self.set_fill_color(200, 220, 255)
        # Title
        self.cell(0, 6, 'For Answer %d : %s' % (num, label), 0, 1, 'L', 1)
        # Line break
        self.ln(4)

    def report_body(self, name):
        # Read text file
        with open(name, 'rb') as fh:
            txt = fh.read().decode('latin-1')
        # Times 12
        self.set_font('Times', '', 12)
        # Output justified text
        self.multi_cell(0, 5, txt, border=1, align = 'J')
        # Line break
        self.ln()
        # Line break
        self.ln()
        # Mention in italics
        self.set_font('', 'I')
        self.cell(0, 5, '(end of excerpt)')

    def report_img(self, num, title, name):

        self.add_page()
        self.titles(num, title)
        self.image(name, h=100,w=190, x=10, y=60, type="png")
        # Line break
        self.ln()

    def print_report(self, num, title, name):
        self.add_page()
        self.titles(num, title)
        self.report_body(name)



pdf = PDF()
pdf.set_title(title)
pdf.set_author('Bhargav Desai')
pdf.print_report(1, 'ANALYSIS', '/tmp/output.txt')
pdf.report_img(1, "Marks Assignment Breakdown Per Point of the Student's Answer",'/tmp/foo.png')
pdf.output('/tmp/tuto3.pdf', 'F')

''

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageChops

fig, ax = plt.subplots(dpi=75, constrained_layout=True)
fig.patch.set_visible(False)
ax.axis('off')
ax.axis('tight')
ax.table(cellText=df.values, colWidths = [1.0]*len(df.columns.values), colLabels=df.columns, cellLoc = 'center').scale(1, 5)
                                                        
fig.tight_layout()
plt.savefig('/tmp/foo.png',bbox_inches='tight')  # Path to save the image
plt.close()


def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

im = Image.open("/tmp/foo.png")
im = trim(im)
im = im.convert('RGB')
im.save("/tmp/foo.png") 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: constrained_layout not applied.  At least one axes collapsed to zero width or height.
  if sys.path[0] == '':


In [ ]:
sim_mat

array([[0.87484391, 0.40950937, 0.50271733, 0.33813908],
       [0.51287723, 0.72144274, 0.30947994, 0.32152879],
       [0.44467748, 0.32708883, 0.91808748, 0.28324439],
       [0.36141643, 0.32238718, 0.3022965 , 0.87905297]])

In [ ]:
ans_key_sent

['Nationalist tensions emerged in the Balkans because of the spread of ideas of romantic nationalism as also the disintegration of the Ottoman Empire that had previously ruled over this area.',
 'The different Slavic communities in the Balkans began to strive for independent rule. They were jealous of each other and every state wanted more territory, even at the expense of others.',
 'Also, the hold of imperial power over the Balkans made the situation worse.',
 'Russia, Germany, England, Austro-Hungary all wanted more control over this area. These conflicts ultimately led to the First World War in 1914.']

In [ ]:
ans_stu_sent

['Patriotic strains developed in the Balkans because of the spread of ideas of romantic nationalism as also the breaking down of the Ottoman Empire that had recently administered over this territory.',
 'All the different Slavic communities in the Balkans werd envious of one another and cach state wanted more area.',
 'Also, the hold of imperial power over the Balkans me worsened the situation.',
 'Russia, Germany, England , Asustro-Hungary all wonted more control over this zone which brought about the first world war in 1914.']

In [ ]:
df

,Student's Point,Associated Weight,Marks Awarded
0,Patriotic strains developed in the Balkans bec...,1.0,0.5
1,All the different Slavic communities in the\nB...,1.0,0.5
2,"Also, the hold of imperial power over the Balk...",1.0,0.5
3,"Russia, Germany, England , Asustro-Hungary all...",1.0,0.5


In [ ]:
df = pd.read_csv('/tmp/out.csv')

In [ ]:
df

,Student's Point,Associated Weight,Marks Awarded
0,Patriotic strains developed in the Balkans bec...,1.0,0.5
1,All the different Slavic communities in the\nB...,1.0,0.5
2,"Also, the hold of imperial power over the Balk...",1.0,0.5
3,"Russia, Germany, England , Asustro-Hungary all...",1.0,0.5


In [ ]:
# Import nltk
import nltk
import spacy
import string

nlp = spacy.load("en_core_web_sm")
ans_stu_sent = convert_answers_to_sentences(v9[0])

In [ ]:
ans_stu_sent

['Two examples to show the contribution of culture to the growth of nationalism : A primary factor for the growth of nationalism in Europe was Romanticism Romanticism was a European culture.',
 "movement aimed at developing national unity by creating a sense of shared heritage and common history The Romantic artist's emphasis on emotions intuition and mystical feelings gave shape and expression to nationalist feelings.",
 '2.',
 'Apart from romanticism language also played an important role in developing nationalist feeling in Europe.',
 'For example during was seen as a symbol of struggle against russians during this period Polish language was forced out of schools and Russian was made compulsory everywhere Mony members of the clergy started refusing to preach in russian and used Polish for church gathtus.',
 'This way there of Polish helped spread the ... message of national unity']

In [ ]:
def handle_abbreviations(sent):

  # better solution for this to come

  sent = sent.replace('St.', 'St')
  sent = sent.replace('Mr.', 'Mr')
  sent = sent.replace('Mrs.', 'Mrs')
  sent = sent.replace('Dr.', 'Dr')
  sent = sent.replace('Hon.', 'Hon')
  sent = sent.replace('Ave.', 'Avenue')
  sent = sent.replace('appx.', 'approx')
  sent = sent.replace('dept.', 'department')
  #sent.translate(str.maketrans('', '', string.punctuation))
  

  return sent

def modify(idx_of_sent, idx_of_word_to_be_replaced, ans_stu_sent, count):

  sent = handle_abbreviations(ans_stu_sent[idx_of_sent]).split()
  word_at_idx_minus_one = sent[idx_of_word_to_be_replaced-1]
  sent.remove(word_at_idx_minus_one)
  sent.insert(idx_of_word_to_be_replaced-1, word_at_idx_minus_one+'.')
  if len(' '.join(sent).split('.')[count-1].split())>=4:
    ans_stu_sent[idx_of_sent] = ' '.join(sent)
  else:
    pass

In [ ]:
def aware_capitalisation(ans_stu_sent):
  
    for idx, sent in enumerate(ans_stu_sent):


        candidates=[]
        not_propernouns=[]
        propernouns = []
        print(sent)
        doc = nlp(sent)
        for token in doc:
          if token.tag_ == 'NNP' or token.tag_ == 'NNPS':
            propernouns.append(token.text)

        print(propernouns)


        for word in sent.split():
    
          if word[0].isupper() and word.translate(str.maketrans('', '', string.punctuation.replace('.', ''))) not in propernouns:
                candidates.append(word)
    
        count=0

        print(candidates)
        words = sent.split()
        for idx2, word in enumerate(words):
                if word in candidates:
                  if not idx2==0:
                    count+=1
                    modify(idx, idx2, ans_stu_sent, count)

                
    print(ans_stu_sent)      

    return ' '.join(ans_stu_sent)

In [ ]:
mod = aware_capitalisation(ans_stu_sent)

Two examples to show the contribution of culture to the growth of nationalism : A primary factor for the growth of nationalism in Europe was Romanticism Romanticism was a European culture.
['Europe', 'Romanticism', 'Romanticism']
['Two', 'A', 'European']
movement aimed at developing national unity by creating a sense of shared heritage and common history The Romantic artist's emphasis on emotions intuition and mystical feelings gave shape and expression to nationalist feelings.
[]
['The', 'Romantic']
2.
[]
[]
Apart from romanticism language also played an important role in developing nationalist feeling in Europe.
['Europe']
['Apart', 'Europe.']
For example during was seen as a symbol of struggle against russians during this period Polish language was forced out of schools and Russian was made compulsory everywhere Mony members of the clergy started refusing to preach in russian and used Polish for church gathtus.
['russians', 'Russian', 'Mony', 'russian', 'Polish']
['For']
This way th

In [ ]:
ans_stu_sent = convert_answers_to_sentences(mod)

In [ ]:
ans_stu_sent

['It was a German National Assembly made upon middle class professionals businessmen artisans belonging to different German regiony.',
 'It was convened on 18 may 1848 in the Church of St Paul frankfurt.',
 'The assembly was responsible for drafting constitution for Germany which for was two-headed by a monarchy ruler.',
 'Be that as if may it confronted resistance from the privileged and military was additionally ruled by the middle class as a result of which it lost its mass support base andreas shut down.']

In [ ]:
doc = nlp('Patriotic strains developed in the Balkans because of the spread of ideas of romantic nationalism as also the breaking down ofthe Ottoman Empire that had recently administered over this territory All the different Slavic communities in the Balkans were envious of one another and each state wanted more area also threshold of imperial power over the Balkans he worsened the situation Russian Germany, England , bistro - Hungary all wonted more control over this zone which brought about the first world war in 1914')
for token in doc:
    print(token.tag_, token.lemma_)

JJ patriotic
NNS strain
VBN develop
IN in
DT the
NNPS Balkans
IN because
IN of
DT the
NN spread
IN of
NNS idea
IN of
JJ romantic
NN nationalism
RB as
RB also
DT the
VBG break
RP down
DT ofthe
NNP Ottoman
NNP Empire
WDT that
VBD have
RB recently
VBN administer
IN over
DT this
NN territory
PDT all
DT the
JJ different
JJ slavic
NNS community
IN in
DT the
NNPS Balkans
VBD be
JJ envious
IN of
NN one
DT another
CC and
DT each
NN state
VBD want
JJR more
NN area
RB also
NN threshold
IN of
JJ imperial
NN power
IN over
DT the
NNPS Balkans
PRP -PRON-
VBD worsen
DT the
NN situation
NNP Russian
NNP Germany
, ,
NNP England
, ,
NNP bistro
HYPH -
NNP Hungary
DT all
VBD wonte
JJR more
NN control
IN over
DT this
NN zone
WDT which
VBD bring
RP about
DT the
JJ first
NN world
NN war
IN in
CD 1914


In [ ]:
import string 

s = 'Germany,'
s.translate(str.maketrans('', '', string.punctuation))

'Germany'

In [ ]:
s.translate(None, string.punctuation)

NameError: ignored

In [ ]:
s

'Patriotic strains developed in the Balkans because of the spread of ideas of romantic nationalism as also the breaking down ofthe Ottoman Empire that had recently administered over this territory All the different Slavic communities in the Balkans were envious of one another and each state wanted more area also threshold of imperial power over the Balkans he worsened the situation russian Germany, England , bistro - Hungary all wonted more control over this zone which brought about the first world war in 1914'

In [ ]:
string.punctuation.replace('.', '')

'!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'